In [54]:
import soundcard as sc
import soundfile as sf
import cv2
import numpy as np

import torch
import torchaudio

from matplotlib import pyplot as plt

SAMPLE_RATE = 48000 # [Hz]. sampling rate.

duration = 200  # Duration of the audio stream in seconds
window_size = 1024  # Size of the FFT window
overlap = 512  # Number of samples to overlap between consecutive windows

with sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True).recorder(samplerate=SAMPLE_RATE) as mic:
    for i in range(duration):
        data = mic.record(numframes=window_size)
        data = torch.from_numpy(data).permute(1, 0)

        print(f"{i} of {duration}", end="\r")

        audio_spectogram = torchaudio.transforms.Spectrogram()(data)
        audio_spectogram = audio_spectogram.log2()[0,:,:].numpy()
        audio_spectogram -= audio_spectogram.min()
        audio_spectogram /= audio_spectogram.max()

        print(audio_spectogram.shape)

        cv2.imshow('stream', audio_spectogram)

        k = cv2.waitKey(33)
        if k==27:    # Esc key to stop
            cv2.destroyAllWindows()
            break
        
        # # closing all open windows
    cv2.destroyAllWindows()

(201, 6)
(201, 6)
(201, 6)
(201, 6)
(201, 6)
(201, 6)
(201, 6)
(201, 6)
(201, 6)
(201, 6)
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
(201, 6)0
